RAG chatbot 
Version 1.0 

Import Libraries

In [2]:
# Imports
from langchain_community.document_loaders import UnstructuredPDFLoader
from langchain_ollama import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_ollama.chat_models import ChatOllama
from langchain_core.runnables import RunnablePassthrough
from langchain_classic.retrievers import MultiQueryRetriever
import chromadb

# Suppress warnings
import warnings
warnings.filterwarnings('ignore')

# Suppress ChromaDB telemetry warnings as they are useless as ChromaDB trying to send usage data but failing because it can't find its own version information
import logging
logging.getLogger("chromadb").setLevel(logging.ERROR)

# Jupyter-specific imports
from IPython.display import display, Markdown

# Set environment variable for protobuf
import os
os.environ["PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION"] = "python"

LOAD PDF

In [3]:
# Load PDF
local_path = "TaskFlow.pdf"
if local_path:
    loader = UnstructuredPDFLoader(file_path=local_path, languages=["deu"])
    data = loader.load()
    print(f"PDF loaded successfully: {local_path}")
else:
    print("Upload documents")

#TODO: language detection!!!!

Could get FontBBox from font descriptor because None cannot be parsed as 4 floats
Could get FontBBox from font descriptor because None cannot be parsed as 4 floats


PDF loaded successfully: TaskFlow.pdf


SPLIT TEXT INTO CHUNKS

In [4]:
#Split text into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.split_documents(data)
print(f"Number of text chunks: {len(chunks)}")

Number of text chunks: 6


CREATE VECTOR DATABASE

In [5]:
# Create vector database
client = chromadb.PersistentClient(path="./chroma_data")
vector_db = Chroma.from_documents(
    documents=chunks,
    embedding=OllamaEmbeddings(model="nomic-embed-text"),
    collection_name="rag_chatbot_collection",
    client=client
)
print("Vector database created successfully.")


Vector database created successfully.


Set up LLM and Retrieval

In [6]:
#Set up LLM and retrieval
local_model = "llama3.2"
llm = ChatOllama(model=local_model, temperature=0.1)

# Query prompt template
QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""You are a query rephrasing assistant for a vector database search.
    Your task is to generate **3** alternative, semantically diverse versions of the user's question. These variations should explore different terminology, structures, or angles of the original query to maximize relevant document retrieval.
    Provide only the rephrased questions, with each one separated by a newline. Do not include any introductory phrases, explanations, or the original question itself.
    Original question: {question}""",
)

# Set up retriever
retriever = MultiQueryRetriever.from_llm(
    vector_db.as_retriever(), 
    llm,
    prompt=QUERY_PROMPT
)